In [1]:
import os
import openai

In [2]:
# Hide this code block
#openai.api_key = 'Add your key here!'

In [3]:
def create_test_prompt(topic,num_questions,num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions}. Each question should have "\
    +f"{num_possible_answers} options. Also include the correct answer for each question using the starting string "\
    +f"Correct Answer:' "
    return prompt

In [45]:
response = openai.Completion.create(engine='gpt-3.5-turbo-instruct',
                                   prompt=create_test_prompt('US History',4,4),
                                   max_tokens=256,
                                   temperature=0.7)

In [46]:
response['choices'][0]['text']

'\n1. Who was the first president of the United States?\nA. John Adams\nB. George Washington\nC. Thomas Jefferson\nD. Abraham Lincoln\nCorrect Answer: B. George Washington\n\n2. The Declaration of Independence was signed in which year?\nA. 1776\nB. 1789\nC. 1792\nD. 1803\nCorrect Answer: A. 1776\n\n3. The Louisiana Purchase was made during the presidency of:\nA. Thomas Jefferson\nB. John Adams\nC. James Madison\nD. George Washington\nCorrect Answer: A. Thomas Jefferson\n\n4. Which war was fought between the Northern and Southern states of the US?\nA. World War I\nB. Revolutionary War\nC. Civil War\nD. War of 1812\nCorrect Answer: C. Civil War'

In [47]:
def create_student_view(test,num_questions):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+'\n'
        else:
                if question_number < num_questions:
                    question_number +=1
                    student_view[question_number] = ''
    return student_view

In [48]:
result = create_student_view(response['choices'][0]['text'],4)

In [49]:
for key in result:
    print(result[key])


1. Who was the first president of the United States?
A. John Adams
B. George Washington
C. Thomas Jefferson
D. Abraham Lincoln


2. The Declaration of Independence was signed in which year?
A. 1776
B. 1789
C. 1792
D. 1803


3. The Louisiana Purchase was made during the presidency of:
A. Thomas Jefferson
B. John Adams
C. James Madison
D. George Washington


4. Which war was fought between the Northern and Southern states of the US?
A. World War I
B. Revolutionary War
C. Civil War
D. War of 1812



In [50]:
def extract_answer(test,num_questions):
    answers = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+'\n'
        
            if question_number < num_questions:
                    question_number +=1
                    answers[question_number] = ''
    return answers

In [51]:
extract_answer(response['choices'][0]['text'],4)

{1: 'Correct Answer: B. George Washington\n',
 2: 'Correct Answer: A. 1776\n',
 3: 'Correct Answer: A. Thomas Jefferson\n',
 4: 'Correct Answer: C. Civil War\n'}

In [52]:
student_view = create_student_view(response['choices'][0]['text'],4)

In [53]:
answers = extract_answer(response['choices'][0]['text'],4)

In [72]:
# TAKE THE EXAM
def take(student_view):
    student_answers= {}
    for question,question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        student_answers[question] = answer
    return student_answers

In [73]:
#student_view.items()

In [74]:
student_answers = take(student_view)


1. Who was the first president of the United States?
A. John Adams
B. George Washington
C. Thomas Jefferson
D. Abraham Lincoln

Enter your answer: b

2. The Declaration of Independence was signed in which year?
A. 1776
B. 1789
C. 1792
D. 1803

Enter your answer: a

3. The Louisiana Purchase was made during the presidency of:
A. Thomas Jefferson
B. John Adams
C. James Madison
D. George Washington

Enter your answer: d

4. Which war was fought between the Northern and Southern states of the US?
A. World War I
B. Revolutionary War
C. Civil War
D. War of 1812

Enter your answer: d


In [75]:
student_answers

{1: 'b', 2: 'a', 3: 'd', 4: 'd'}

In [76]:
answers

{1: 'Correct Answer: B. George Washington\n',
 2: 'Correct Answer: A. 1776\n',
 3: 'Correct Answer: A. Thomas Jefferson\n',
 4: 'Correct Answer: C. Civil War\n'}

In [77]:
def grade(correct_answer_dict,student_answers):
    correct_answers = 0
    for question,answer in student_answers.items():
        if answer.upper() == correct_answer_dict[question][16]:
            correct_answers +=1
    grade = 100*correct_answers / len(answers)
            
    if grade < 60:
        passed = "NO PASS"
    else:
        passed = "PASS!"
                
    return f"{correct_answers}/{len(answers)} correct! You got {grade} grade, {passed}"

In [78]:
grade(answers, student_answers)

'2/4 correct! You got 50.0 grade, NO PASS'